# HW2 – Optimization

>Quick note:
Comparasion and Python code is not completed, I know that is not exaxctly what was asked in the assignment. But it was just a lot in short time for me.

## Part 1

### Problem 1

In [3]:
import jax
import jax.numpy as jnp
from jax import grad, random

def J(U, V, Y, lambda_val):
    return jnp.linalg.norm(jnp.dot(U, V) - Y, 'fro')**2 + lambda_val**2 * (jnp.linalg.norm(U, 'fro')**2 + jnp.linalg.norm(V, 'fro')**2)

grad_J_U = grad(J, argnums=0)  # Gradient with respect to U
grad_J_V = grad(J, argnums=1)  # Gradient with respect to V

# Examples
n, k = 5, 3
lambda_val_example = 0.5

key = random.PRNGKey(0)

U_example = random.normal(key, (n, k))
V_example = random.normal(key, (k, n))
Y_example = random.normal(key, (n, n))

grad_U_example = grad_J_U(U_example, V_example, Y_example, lambda_val_example)
grad_V_example = grad_J_V(U_example, V_example, Y_example, lambda_val_example)

print("Gradient with respect to U:\n", grad_U_example)
print("Gradient with respect to V:\n", grad_V_example)


Gradient with respect to U:
 [[  4.3180423  18.366312    9.453627 ]
 [  7.372599   -6.975644  -14.219867 ]
 [  8.444608   -9.701819  -12.073999 ]
 [  2.1707811  12.403317   13.254414 ]
 [  2.8677337 -16.268196  -22.240843 ]]
Gradient with respect to V:
 [[ -0.8207928    0.17444533   3.725928     9.408623     5.452488  ]
 [-16.901579    18.467587   -17.322805   -21.162186   -14.764771  ]
 [ -4.9798985   12.033523   -15.780456   -22.070341   -21.243605  ]]


Given the function
$ J(U, V) = \| UV - Y \|_F^2 + \frac{\lambda}{2} (\| U \|_F^2 + \| V \|_F^2) $

Gradient with respect to $ U $:

1. Differentiate the first term $ \| UV - Y \|_F^2 $ with respect to $ U $:
   $ \frac{\partial}{\partial U} \| UV - Y \|_F^2 = \frac{\partial}{\partial U} \text{Tr}((UV - Y)^T(UV - Y)) $
   Using the properties of trace and derivative, we get:
   $ = 2(UV - Y)V^T $

2. Differentiate the regularization term $ \frac{\lambda}{2} \| U \|_F^2 $ with respect to $ U $:
   $ \frac{\partial}{\partial U} \frac{\lambda}{2} \| U \|_F^2 = \lambda U $

Combining these, the gradient with respect to $ U $ is:
$ \nabla_U J(U, V) = 2(UV - Y)V^T + \lambda U $

Gradient with respect to $ V $:

1. Differentiate the first term $ \| UV - Y \|_F^2 $ with respect to $ V $:
   $ \frac{\partial}{\partial V} \| UV - Y \|_F^2 = \frac{\partial}{\partial V} \text{Tr}((UV - Y)^T(UV - Y)) $
   Similarly, we get:
   $ = 2U^T(UV - Y) $

2. Differentiate the regularization term $ \frac{\lambda}{2} \| V \|_F^2 $ with respect to $ V $:
   $ \frac{\partial}{\partial V} \frac{\lambda}{2} \| V \|_F^2 = \lambda V $

Combining these, the gradient with respect to $ V $ is:
$ \nabla_V J(U, V) = 2U^T(UV - Y) + \lambda V $

These gradients are used in optimization to find the values of $ U $ and $ V $ that minimize $ J(U, V) $.


### Problem 2

Given the function
$ f(\mathbf{w}) = \sum_{i=1}^{m} \log(1 + e^{y_i \mathbf{w}^T \mathbf{x}_i}) + \frac{1}{2} \| \mathbf{w} \|_2^2 $

Gradient of $ f(\mathbf{w}) $:

The gradient is given by:
$ \nabla f(\mathbf{w}) = \sum_{i=1}^{m} \frac{y_i \mathbf{x}_i e^{y_i \mathbf{w}^T \mathbf{x}_i}}{1 + e^{y_i \mathbf{w}^T \mathbf{x}_i}} + \mathbf{w} $

Hessian of $ f(\mathbf{w}) $:

The Hessian is given by:
$ H(f(\mathbf{w})) = \sum_{i=1}^{m} \frac{y_i^2 \mathbf{x}_i \mathbf{x}_i^T e^{y_i \mathbf{w}^T \mathbf{x}_i}}{(1 + e^{y_i \mathbf{w}^T \mathbf{x}_i})^2} + I $
where $ I $ is the identity matrix.


In [10]:
import jax.numpy as jnp
from jax import grad, jit, hessian, random

def f(w, X, y):
    # X is a matrix of shape (m, n) where each row is x_i
    # y is a vector of length m
    log_terms = jnp.log1p(jnp.exp(jnp.dot(X, w) * y))
    regularization = 0.5 * jnp.sum(w**2)
    return jnp.sum(log_terms) + regularization

grad_f = grad(f, argnums=0)

hessian_f = hessian(f, argnums=0)

m, n = 10, 5  # Example dimensions
key = random.PRNGKey(0) 
w_example = random.normal(key, (n,)) 
X_example = random.normal(key, (m, n)) 
y_example = random.normal(key, (m,))

grad_f_example = grad_f(w_example, X_example, y_example)
hessian_f_example = hessian_f(w_example, X_example, y_example)

print("Gradient:\n", grad_f_example)
print("Hessian:\n", hessian_f_example)


Gradient:
 [ 0.10853742 -1.8221643  -0.919782    1.1513234   0.26755393]
Hessian:
 [[ 1.8347886   0.16822092 -0.48792958 -0.17042848  0.05137864]
 [ 0.16822092  1.2593102  -0.00693707  0.0586279   0.03631498]
 [-0.48792958 -0.00693707  1.4965051   0.25343034 -0.08942212]
 [-0.17042848  0.0586279   0.25343034  1.3799262  -0.00960475]
 [ 0.05137864  0.03631499 -0.08942211 -0.00960474  1.3466067 ]]


### Problem 3

Given the function
$ g(\mathbf{x}) = f(\mathbf{A}\mathbf{x} + \mathbf{b}) $

where $ \nabla f(\mathbf{y}) $ denotes the gradient of $ f $ at any point $ \mathbf{y} $, the gradient of $ g $ with respect to $ \mathbf{x} $ is given by:

$ \nabla g(\mathbf{x}) = \mathbf{A}^T \nabla f(\mathbf{A}\mathbf{x} + \mathbf{b}) $

## Part 2

### Problem 1

Given the function
$ g(\mathbf{x}) = f(\mathbf{A}\mathbf{x} + \mathbf{b}) $

where $ \nabla f(\mathbf{y}) $ denotes the gradient of $ f $ at any point $ \mathbf{y} $, the gradient of $ g $ with respect to $ \mathbf{x} $ is given by:

$ \nabla g(\mathbf{x}) = \mathbf{A}^T \nabla f(\mathbf{A}\mathbf{x} + \mathbf{b}) $

### Problem 2

(a) Gradient of $ f(X) = \sum_{i=1}^{n} \lambda_i(X) $:

Since the sum of the eigenvalues $ \lambda_i(X) $ of a matrix $ X $ is equal to its trace:
$ f(X) = \text{Tr}(X) $

The gradient of the trace of a matrix with respect to the matrix is the identity matrix:
$ \nabla_X f(X) = I $
where $ I $ is the identity matrix of the same size as $ X $.


(b) Gradient of $ f(X) = \prod_{i=1}^{n} \lambda_i(X) $:

The product of the eigenvalues $ \lambda_i(X) $ of a matrix $ X $ is equal to its determinant:
$ f(X) = \det(X) $

For a matrix $ X $ with distinct eigenvalues, the gradient of the determinant can be expressed in terms of the adjugate of $ X $:
$ \nabla_X f(X) = \text{adj}(X)^T $


In [4]:
import jax.numpy as jnp
from jax import grad, jit, vmap

def softmax(w):
    w_max = jnp.max(w)
    w_stable = w - w_max
    e_w = jnp.exp(w_stable)
    return e_w / jnp.sum(e_w)

def f(w):
    return softmax(w)

jacobi_matrix = vmap(grad(f), in_axes=(0,))

w_example = jnp.array([1.0, 2.0, 3.0])

jacobi_matrix_example = jacobi_matrix(w_example)


In [7]:
def sum_of_eigenvalues(X):
    eig_vals, _ = jnp.linalg.eigh(X)
    return jnp.sum(eig_vals)

grad_sum_eigenvalues = grad(sum_of_eigenvalues)

X_example = jnp.array([[1.0, 2.0], [2.0, 3.0]])

grad_sum_eigenvalues_example = grad_sum_eigenvalues(X_example)
grad_sum_eigenvalues_example


Array([[ 1.0000001e+00, -4.8470916e-09],
       [-4.8470916e-09,  1.0000001e+00]], dtype=float32)

In [8]:
def product_of_eigenvalues(X):
    eig_vals, _ = jnp.linalg.eigh(X)
    return jnp.prod(eig_vals)

grad_product_eigenvalues = grad(product_of_eigenvalues)

grad_product_eigenvalues_example = grad_product_eigenvalues(X_example)
grad_product_eigenvalues_example


Array([[ 3., -2.],
       [-2.,  1.]], dtype=float32)